In [ ]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import logging, sys
import pandas as pd
import statsmodels.api as sm
from os import path
from volcano_ash import *
_LOG = logging.getLogger(__name__)
stdout_hdlr = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('[%(asctime)s.%(msecs)03d - %(levelname)s] %(message)s')
stdout_hdlr.setFormatter(formatter)
_LOG.addHandler(stdout_hdlr)
_LOG.setLevel(logging.DEBUG)

In [ ]:
path_list = np.arange(118, 86, -1)
row_list = np.arange(64, 94)

In [ ]:
change_erupt = np.zeros([row_list.shape[0], path_list.shape[0]])
for path_id in path_list:
    for row_id in row_list:
        pathrow_id = '0'.join([str(path_id), str(row_id)])
        csv_fname = 'threshold/' + pathrow_id + ".csv"
        if not path.exists(csv_fname):
            continue
        df = pd.read_csv(csv_fname) 
        if (df.shape[0] <= 5):
            continue
        df = df.set_index("time")
        df.index = pd.DatetimeIndex(df.index)   
        after_erupt = df[(df.index >= np.datetime64('1991-06-15', 'D')) & (df.index < np.datetime64('1994-01-01', 'D'))].max()
        change =  after_erupt - df[df.index < np.datetime64('1991-06-15', 'D')].min()
        #change_erupt[np.where(row_list == row_id)[0], np.where(path_list == path_id)[0]] = change / after_erupt
        change_erupt[np.where(row_list == row_id)[0], np.where(path_list == path_id)[0]] = change

In [ ]:
change_erupt[change_erupt == 0] = np.nan

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
im = ax.imshow(np.ma.masked_where(np.isnan(change_erupt), np.clip(change_erupt, 0, 550)),
               extent=[path_list[0]+0.5, path_list[-1]-0.5, row_list[-1]+0.5, row_list[0]-0.5])
ax.xaxis.set_ticks(path_list)
ax.yaxis.set_ticks(row_list)
plt.colorbar(im);
plt.title("Absolute impact magnitude")
plt.grid()

In [ ]:
abnormal_list = np.zeros([row_list.shape[0], path_list.shape[0]]).astype("datetime64[D]")
reset_list = np.zeros([row_list.shape[0], path_list.shape[0]]).astype("datetime64[D]")
#standard_list = []
for path_id in path_list:
    for row_id in row_list:
        pathrow_id = '0'.join([str(path_id), str(row_id)])
        csv_fname = 'threshold/' + pathrow_id + ".csv"
        if not path.exists(csv_fname):
            continue
        df = pd.read_csv(csv_fname)
        if (df.shape[0] <= 5):
            continue
        months, monthly_median, monthly_threshold = stats_by_month(df)
        if len(months) < 3:
            continue
        if max(monthly_median) >= 650:
            continue
        standard = np.array([months, monthly_threshold])
        abnormal_start, normal_reset = month_ppnormal(df, standard, 200, 60)
        if abnormal_start is None or normal_reset is None:
            continue
        abnormal_list[np.where(row_list == row_id)[0], np.where(path_list == path_id)[0]] = abnormal_start
        reset_list[np.where(row_list == row_id)[0], np.where(path_list == path_id)[0]] = normal_reset
        #standard_list.append(standard)

In [ ]:
time_period = (reset_list - abnormal_list) / np.timedelta64(1, "D")
time_period[time_period == time_period.max()] = np.nan
time_period[time_period == 0] = np.nan

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
im = ax.imshow(time_period, extent=[path_list[0]+0.5, path_list[-1]-0.5, row_list[-1]+0.5, row_list[0]-0.5])
ax.xaxis.set_ticks(path_list)
ax.yaxis.set_ticks(row_list)
plt.colorbar(im);
plt.grid()

In [ ]:
path_id = '98'
row_id = '068'

In [ ]:
csv_fname = 'threshold/' + path_id + row_id + ".csv"
df = pd.read_csv(csv_fname)
months, monthly_median, monthly_threshold = stats_by_month(df)
standard = np.array([months, monthly_threshold])
abnormal_start, normal_reset = month_ppnormal(df, standard, 200,60)

In [ ]:
df = df.set_index("time")
df.index = pd.DatetimeIndex(df.index)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16,9))
threshold_all = []
#ax.plot(months, monthly_threshold, "*-", label='threshold', color='r')
for y in df.index.year.unique():
    if y < 1990:
        continue
    if y > 1996:
        break
    threshold_all += [np.datetime64(str(y) + '-0' + str(m), 'M')
             if len(str(m)) == 1 else np.datetime64(str(y) + '-' + str(m), 'M')for m in months]
ax.plot(df[(df.index.year >= 1990) & (df.index.year <=1996)], 'o--', label='darkest mean')
ax.plot(threshold_all, monthly_threshold * (1996-1990+1), '*-', color='red', label='threshold')
if abnormal_start is not None:
    ax.vlines([abnormal_start, normal_reset], df.min(), df.max(), color='green', label='impact duration')
plt.xticks(threshold_all, threshold_all, rotation='vertical')
plt.grid()
plt.title('path/row '+path_id + row_id)
ax.legend()